#### Install and import Python libraries

In [ ]:
import os
import sys
import time
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import motmetrics as mm

mm.lap.default_solver = "lap"
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
root_dir = ".."
sys.path.append(os.path.join(root_dir, "src"))

In [ ]:
from market.models import build_model
from mot.data.data_track import MOT16Sequences
from mot.models.object_detector import FRCNN_FPN
from mot.tracker.advanced import ReIDHungarianTracker
from mot.tracker.base import Tracker
from mot.visualize import plot_sequence
from mot.eval import evaluate_mot_accums, get_mot_accum


# Object detector

The object detector is a [Faster R-CNN](https://arxiv.org/abs/1506.01497) with a Resnet50 feature extractor. We trained the native PyTorch implementation of Faster-RCNN. For more information check out the corresponding PyTorch [webpage](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html).


## Configuration

In [ ]:
obj_detect_model_file = os.path.join(root_dir, "models/faster_rcnn_fpn.model")
obj_detect_nms_thresh = 0.3

In [ ]:
device = torch.device("cuda:2") if torch.cuda.is_available() else torch.device("cpu")

# object detector
obj_detect = FRCNN_FPN(num_classes=2, nms_thresh=obj_detect_nms_thresh)
obj_detect_state_dict = torch.load(
    obj_detect_model_file, map_location=lambda storage, loc: storage
)
obj_detect.load_state_dict(obj_detect_state_dict)
obj_detect.eval()
obj_detect = obj_detect.to(device)

# Multi-object tracking

## Configuration

In [ ]:
seed = 12345
seq_name = "MOT16-reid"  # We recommend to use this subset.
data_dir = os.path.join(root_dir, "data/MOT16")
output_dir = os.path.join(root_dir, "output")

## Setup

In [ ]:
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

# dataset
sequences = MOT16Sequences(seq_name, data_dir)

# Reid + Hungarian algo+ cosine dist

In this part, we combine the ReID model with the tracker to compute appearance-based matching.

Since we're using a very simple ReID model, tracking only based on apperance is very difficult.
For all pairs of tracks and detections, the distance is set to `_UNMATCHED_COST`, when the IoU value is below 0.5. For all other cases, the costs are based on the cosine distance. 
The cosine distance should be between -1 and 1.

## reid model

In [ ]:
model_path = root_dir + "/models/resnet50_reid.pth"

reid_model = build_model("resnet34", 751, loss="softmax", pretrained=True)
reid_ckpt = torch.load(model_path)
reid_model.load_state_dict(reid_ckpt)

In [ ]:
reid_model = reid_model.to(device)

### Run tracker 


In [ ]:
# Old Trackers
# tracker = BaseTrackerIoU(obj_detect)
# tracker = HungarianIoUTracker(obj_de|tect)

tracker = ReIDHungarianTracker(reid_model=reid_model, obj_detect=obj_detect)

In [ ]:
time_total = 0
mot_accums = []
results_seq = {}
for seq in sequences:
    tracker.reset()
    now = time.time()

    print(f"Tracking: {seq}")

    data_loader = DataLoader(seq, batch_size=1, shuffle=False)

    for frame in tqdm(data_loader):
        tracker.step(frame)
    results = tracker.get_results()
    results_seq[str(seq)] = results

    if seq.no_gt:
        print(f"No GT evaluation data available.")
    else:
        mot_accums.append(get_mot_accum(results, seq))

    time_total += time.time() - now

    print(f"Tracks found: {len(results)}")
    print(f"Runtime for {seq}: {time.time() - now:.1f} s.")

    # seq.write_results(results, os.path.join(output_dir))

print(f"Runtime for all sequences: {time_total:.1f} s.")
if mot_accums:
    evaluate_mot_accums(
        mot_accums, [str(s) for s in sequences if not s.no_gt], generate_overall=True
    )

## Baseline Tracker Results

            IDF1   IDP   IDR   Rcll  Prcn  GT  MT  PT ML  FP   FN IDs   FM  MOTA  MOTP
    MOT16-02 32.2% 49.8% 23.8% 30.8% 64.4%  62  5  22 35 3170 12858  52   93 13.5% 0.086
    MOT16-05 47.7% 53.9% 42.8% 57.8% 72.7% 133 39  64 30 1502  2917  87  103 34.9% 0.144
    MOT16-09 43.0% 48.8% 38.4% 51.9% 66.1%  26  7  14  5 1420  2559  39   66 24.5% 0.107
    MOT16-11 49.0% 54.1% 44.8% 55.8% 67.5%  75 15  32 28 2542  4166  20   39 28.7% 0.080
    OVERALL  41.0% 51.8% 33.9% 44.1% 67.3% 296 66 132 98 8634 22500 198  301 22.2% 0.101

## Hungarian Tracker Results

             IDF1    IDP  IDR  Rcll  Prcn   GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP 
    MOT16-02 39.1% 55.5% 30.2% 52.3% 96.2%  62  11  38 13  383  8870 246  215 48.9% 0.096 
    MOT16-05 55.1% 65.2% 47.7% 68.8% 94.2% 133  55  66 12  295  2158 199  155 61.7% 0.143  
    MOT16-09 50.2% 62.0% 42.1% 66.4% 97.8%  26  13  12  1   80  1789  76   78 63.5% 0.083  
    MOT16-11 60.4% 66.6% 55.3% 80.2% 96.6%  75  42  26  7  266  1868  99   86 76.3% 0.083  
    OVERALL  49.0% 61.5% 40.6% 63.5% 96.2% 296 121 142 33 1024 14685 620  534 59.4% 0.099 

# ReidHungarianIoU Tracker results

             IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP IDt IDa IDm
    MOT16-02 38.8% 55.1% 29.9% 52.3% 96.2%  62  11  38 13  383  8870 260  215 48.8% 0.096 119 146  11
    MOT16-05 55.7% 66.0% 48.2% 68.8% 94.2% 133  56  65 12  295  2158 185  150 61.9% 0.142  69 138  24
    MOT16-09 50.2% 62.0% 42.1% 66.4% 97.8%  26  13  12  1   80  1789  76   78 63.5% 0.083  23  58   5
    MOT16-11 62.8% 69.2% 57.5% 80.2% 96.6%  75  42  26  7  266  1868  87   86 76.5% 0.083  25  68   9
    OVERALL  49.5% 62.3% 41.1% 63.5% 96.2% 296 122 141 33 1024 14685 608  529 59.5% 0.099 236 410  49

## Visualize tracking results

In [ ]:
plot_sequence(
    results_seq["MOT16-02"],
    [s for s in sequences if str(s) == "MOT16-02"][0],
    first_n_frames=3,
)